### --- Day 15: Beacon Exclusion Zone ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/15

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2022, 15);
WriteLines(inputLines);

Loading puzzle file: Day15.txt
Total lines: 28
Max line length: 73

Sensor at x=3088287, y=2966967: closest beacon is at x=3340990, y=2451747
Sensor at x=289570, y=339999: closest beacon is at x=20077, y=1235084
Sensor at x=1940197, y=3386754: closest beacon is at x=2010485, y=3291030
Sensor at x=1979355, y=2150711: closest beacon is at x=1690952, y=2000000
Sensor at x=2859415, y=1555438: closest beacon is at x=3340990, y=2451747


In [4]:
string[] testInputLines = 
[
    "Sensor at x=2, y=18: closest beacon is at x=-2, y=15",
    "Sensor at x=9, y=16: closest beacon is at x=10, y=16",
    "Sensor at x=13, y=2: closest beacon is at x=15, y=3",
    "Sensor at x=12, y=14: closest beacon is at x=10, y=16",
    "Sensor at x=10, y=20: closest beacon is at x=10, y=16",
    "Sensor at x=14, y=17: closest beacon is at x=10, y=16",
    "Sensor at x=8, y=7: closest beacon is at x=2, y=10",
    "Sensor at x=2, y=0: closest beacon is at x=2, y=10",
    "Sensor at x=0, y=11: closest beacon is at x=2, y=10",
    "Sensor at x=20, y=14: closest beacon is at x=25, y=17",
    "Sensor at x=17, y=20: closest beacon is at x=21, y=22",
    "Sensor at x=16, y=7: closest beacon is at x=15, y=3",
    "Sensor at x=14, y=3: closest beacon is at x=15, y=3",
    "Sensor at x=20, y=1: closest beacon is at x=15, y=3",
];

In [5]:
/// <summary>
/// Explore all points for given Manhattan distance
/// </summary>
IEnumerable<Point>Explore(int distance)
{
    var yDist = (int x) => distance - Math.Abs(x);

    foreach (var x in Enumerable.Range(-distance, distance * 2 + 1))
    foreach (var y in Enumerable.Range(-yDist(x), yDist(x) * 2 + 1))
    {
        yield return (x, y);
    }
}

In [6]:
int Distance(int a, int b) => Math.Abs(a - b);
int Distance(Point a, Point b) => Distance(a.X, b.X) + Distance(a.Y, b.Y);

In [7]:
int FindNoBeacons(string[] inputLines, int checkRow)
{
    HashSet<Point> noBeaconSet = new();

    foreach (var line in inputLines[..])
    {
        var (x, y, a, b) = line.ParseInts().ToArray();

        Point sensor = (x, y);
        Point beacon = (a, b);

        var distance = Distance(sensor, beacon);

        var noBeacons = Explore(distance)
            .Select(p => sensor + p)
            .Where(p => p != beacon)
            .Where(p => p.Y == checkRow);

        noBeaconSet.UnionWith(noBeacons);
    }
    
    return noBeaconSet.Count;
}

var testAnswer = FindNoBeacons(testInputLines, 10);
Console.WriteLine(testAnswer);

26


In [8]:
// This takes too long to run!

// var part1Answer = FindNoBeacons(inputLines, 2000000);
// Console.WriteLine(part1Answer);

In retrospect, the above takes way too long as it calculates the entire "exploration diamond" when we only care about a specific line. Some of the sensors may not even reach the line in question. I guess we need to rewrite to explore the target line only. Let's try again...

In [9]:
int FindNoBeaconLine(string[] inputLines, int targetLine)
{
    HashSet<Point> noBeacons = new();
    HashSet<Point> hardware = new();

    foreach (var line in inputLines)
    {
        var (x, y, a, b) = line.ParseInts().ToArray();

        Point sensor = (x, y);
        Point beacon = (a, b);

        hardware.Add(beacon);

        var distance = Distance((x, y), (a, b));
        var ySteps = Distance(y, targetLine);

        // Distance too far?
        if (ySteps > distance) { continue; }

        var xSteps = distance - ySteps;
        var xRange = Enumerable.Range(-xSteps, 2 * xSteps + 1);
        var noBeaconLine = xRange.Select(r => new Point(x + r, targetLine));

        noBeacons.UnionWith(noBeaconLine);
    }

    noBeacons.ExceptWith(hardware);

    return noBeacons.Count();
}

In [10]:
// In this example, in the row where y=10, there are 26 positions...

var testAnswer2 = FindNoBeaconLine(testInputLines, 10);
Console.WriteLine(testAnswer2);

26


In [11]:
// Consult the report from the sensors you just deployed. In the row where
// y=2000000, how many positions cannot contain a beacon?

var part1Answer = FindNoBeaconLine(inputLines, targetLine: 2000000);
Console.WriteLine(part1Answer);

5142231


In [12]:
// 5142231 is correct!
Ensure(5142231, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/15

Hmm, this seems like another case where the simplest approach will be too computationally expensive. I don't think applying our row-wise approach from part 1 will be efficient enough for 4,000,000 rows in part 2.

From our experience with other AoC puzzles, often this means there is some kind of aggregate approach. In this case, we know that each sensor's effect on a given row is always a contiguous range. Perhaps we can merge these ranges together until they fill the complete row, in which case we know the distress beacon is not there. Likewise we know that beacon row will be two ranges split at exactly one point.

Let's start by modelling the range:

In [14]:
record Range
{
    public Range(int from, int to)
    {
        (From, To) = from < to ? (from, to) : (to, from);
    }

    public int From { get; private set; }
    public int To { get; private set; }

    public int Count => To - From + 1;

    public override string ToString() => $"{From} -> {To}";

    public bool TryMerge(Range other, out Range result)
    {
        var (left, right) = Order(this, other);

        if (left.To + 1 < right.From)
        {
            result = default;
            return false;
        }

        result = new(left.From, Math.Max(left.To, right.To));
        return true;
    }

    public static (Range left, Range Right) Order(Range a, Range b)
    {
        return a.From <= b.From ? (a, b) : (b, a);
    }
}

In [15]:
// To isolate the distress beacon's signal, you need to determine its tuning
// frequency, which can be found by multiplying its x coordinate by 4000000 and
// then adding its y coordinate.

static long TuningFrequency(Point p) => 4000000L * p.X + p.Y;

In [16]:
long FindDistressBeacon(string[] inputLines, int targetMax)
{
    List<Range>[] rowRanges = new List<Range>[targetMax + 1];

    foreach (var line in inputLines)
    {
        var (x, y, a, b) = line.ParseInts().ToArray();

        Point sensor = (x, y);
        Point beacon = (a, b);
        var distance = Distance(sensor, beacon);

        for (var targetLine = 0; targetLine <= targetMax; targetLine++)
        {
            var ySteps = Distance(y, targetLine);

            // Distance too far?
            if (ySteps > distance) { continue; }

            var xSteps = distance - ySteps;
            Range xRange = new(
                from: Math.Max(sensor.X - xSteps, 0),
                to: Math.Min(sensor.X + xSteps, targetMax)
            );

            var rowRange = rowRanges[targetLine] ??= new();
            foreach (var r in rowRange.ToList())
            {
                if (xRange.TryMerge(r, out var merged))
                {
                    rowRange.Remove(r);
                    xRange = merged;
                }
            }
            rowRange.Add(xRange);
        }
    }

    long result = -1;
    for (var y = 0; y < rowRanges.Length; y++)
    {
        var rangeRow = rowRanges[y];

        if (rangeRow is [var a, var b])
        {
            // The split we're looking for
            var x = a switch {
                { From: 0, To: var t } => t + 1,
                { From: var f } => f - 1
            };
            result = TuningFrequency((x, y));
            break;
        }

        // For now, ignore the unlikely possibility of the beacon being at the
        // start / end of the row!
    }

    return result;
}

In [17]:
// In the example above... The tuning frequency for this distress beacon is
// 56000011.

var part2TestAnswer = FindDistressBeacon(testInputLines, targetMax: 20);
Console.WriteLine(part2TestAnswer);

56000011


In [18]:
// Find the only possible position for the distress beacon. What is its tuning
// frequency?

var part2Answer = FindDistressBeacon(inputLines, targetMax: 4000000);
Console.WriteLine(part2Answer);

10884459367718


In [19]:
// 10884459367718 is correct!
Ensure(10884459367718L, part2Answer);

## Exploration

Looks like our approach for part 2 worked, but are there any other approaches I missed? Looking at Peter Norvig's solution, he had the insight that the distress beacon will always reside on the perimeter of the sensors' ranges. 

I thought I could try a set-based solution whereby we take the set of edge points __on__ the perimeter of the sensors, then test the points outside the perimeter- if it is bordered on all four sides by edges, it must be the distress beacon. Whilst it is true that the distress beacon must be bordered by edges on all sides, it turns out this returns too many false positives, as sensor ranges can overlap, resulting in many points bordered on all sides by edges.

For example here are the edges of the test input:

```
......#...#.#.#.....
.....#.....#.#.#....
....#.....#.#.#.#...
...#.....#.#.#.#.#..
..#.....#...#.#.#.#.
.#.....#.....#.#.#.#
#.....#.....#...#.#.
.....#.....#.....#.#
#...#.......#...#.#.
.#.#.........#.#.#.#
..#.........#.#.#.#.
..##.......#.#.#.#..
.####.....#.#.#.#...
##..##...#.#.#.#....
#....##.#.#.#...##..
......##.#.#.#.####.
.......##.#...##..##
........##.#.###...#
........###.##..#.#.
.......##..###...#..
```

Having scrapped this approach, instead I'll try Peter's approach of testing all the perimeter points and excluding those within the range of any sensor - the remaining point is the distress beacon!

In [20]:
IEnumerable<Point> Perimeter(Point centroid, int distance)
{
    var trace = centroid + (Up * distance);
    Point[] directions = [Down + Right, Down + Left, Up + Left, Up + Right];

    foreach (var dir in directions)
    foreach (var _ in Enumerable.Range(0, distance))
    {
        trace += dir;
        yield return trace;
    } 
}

In [21]:
IEnumerable<Point> Perimeter(Point sensor, Point beacon)
    => Perimeter(sensor, Distance(sensor, beacon) + 1);

long SolvePart2ViaParameter(string[] inputLines, int rangeMax)
{
    var sbq = from l in inputLines
              let ints = l.ParseInts().ToArray()
              let sensor = new Point(ints[0], ints[1])
              let beacon = new Point(ints[2], ints[3])
              let distance = Distance(sensor, beacon)
              select (sensor, beacon, distance);
    var sensorBeacons = sbq.ToList();

    var pq = from sb in sensorBeacons
             from p in Perimeter(sb.sensor, sb.beacon)
             where p.X >= 0 && p.X <= rangeMax
             where p.Y >= 0 && p.Y <= rangeMax
             where !sensorBeacons.Any(sb2 => Distance(sb2.sensor, p) <= sb2.distance)
             select p;
    return TuningFrequency(pq.First());
}

In [22]:
var part2TestAnswer2 = SolvePart2ViaParameter(testInputLines, 20);
Console.WriteLine(part2TestAnswer2);
Ensure(56000011, part2TestAnswer2);

var part2Answer2 = SolvePart2ViaParameter(inputLines, 4000000);
Console.WriteLine(part2Answer2);
Ensure(10884459367718L, part2Answer2);

56000011
10884459367718


Final comment: originally I wrote `SolvePart2ViaParameter` as a series of `foreach` loops, and the runtime was around 10 seconds. For aesthetic reasons I rewrote it as a couple of LINQ expressions and runtime dropped to under a second! I was super surprised by this- rarely have I ever experienced _any_ runtime improvement when rewriting to LINQ expressions, let alone a tenfold improvement! I am curious to look into why, but I'll force myself to park it for now and move on to the next puzzle 😊 